In [1]:
import gym
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from collections import deque
print("Gym:", gym.__version__)
print("Tensorflow:", tf.__version__)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Instructions for updating:
non-resource variables are not supported in the long term
Gym: 0.25.0
Tensorflow: 2.9.1


In [2]:
env_name = "CartPole-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Action space: Discrete(2)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead

In [3]:
class QNetwork():
    def __init__(self, state_dim, action_size):
        self.state_in = tf.placeholder(tf.float32, shape=[None, *state_dim])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.q_target_in = tf.placeholder(tf.float32, shape=[None])
        action_one_hot = tf.one_hot(self.action_in, depth=action_size)

        self.hidden1 = tf.layers.dense(self.state_in, 100, activation=tf.nn.relu)
        self.q_state = tf.layers.dense(self.hidden1, action_size, activation=None)
        self.q_state_action = tf.reduce_sum(tf.multiply(self.q_state, action_one_hot), axis=1)

        self.loss = tf.reduce_mean(tf.square(self.q_state_action - self.q_target_in))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss)

    def update_model(self, session, state, action, q_target):
        feed = {self.state_in: state, self.action_in: action, self.q_target_in: q_target}
        session.run(self.optimizer, feed_dict=feed)

    def get_q_state(self, session, state):
        q_state = session.run(self.q_state, feed_dict={self.state_in: state})
        return q_state

In [4]:
class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = deque(maxlen=maxlen)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        sample_size = min(len(self.buffer), batch_size)
        samples = random.choices(self.buffer, k=sample_size)
        return map(list, zip(*samples))

In [5]:
class DQNAgent():
    def __init__(self, env):
        self.state_dim = env.observation_space.shape
        self.action_size = env.action_space.n
        self.q_network = QNetwork(self.state_dim, self.action_size)
        self.replay_buffer = ReplayBuffer(maxlen=10000)
        self.gamma = 0.97
        self.eps = 1.0

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def get_action(self, state):
        q_state = self.q_network.get_q_state(self.sess, [state])
        action_greedy = np.argmax(q_state)
        action_random = np.random.randint(self.action_size)
        action = action_random if random.random() < self.eps else action_greedy
        return action

    def train(self, state, action, next_state, reward, done):
        self.replay_buffer.add((state, action, next_state, reward, done))
        states, actions, next_states, rewards, dones = self.replay_buffer.sample(50)
        q_next_states = self.q_network.get_q_state(self.sess, next_states)
        q_next_states[dones] = np.zeros([self.action_size])
        q_targets = rewards + self.gamma * np.max(q_next_states, axis=1)
        self.q_network.update_model(self.sess, states, actions, q_targets)

        if done: self.eps = max(0.1, 0.99*self.eps)

    def __del__(self):
        self.sess.close()

In [6]:
agent = DQNAgent(env)
num_episodes = 400

for ep in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        print(next_state)
        agent.train(state, action, next_state, reward, done)
        env.render()
        total_reward += reward
        state = next_state

    print("Episode: {}, total_reward: {:.2f}".format(ep, total_reward))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  if __name__ == "__main__":
2022-07-30 22:05:41.479140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-30 22:05:41.491197: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


[-0.02689504 -0.17883457 -0.01860823  0.26799107]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/core.py:58: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  "You are calling render method, "


[-0.03047173 -0.37368608 -0.01324841  0.5547472 ]
[-0.03794545 -0.17838064 -0.00215346  0.25791985]
[-0.04151307 -0.37347177  0.00300494  0.54992276]
[-0.0489825  -0.17839216  0.01400339  0.25818813]
[-0.05255035 -0.3737112   0.01916715  0.5552548 ]
[-0.06002457 -0.569097    0.03027225  0.8539144 ]
[-0.07140651 -0.76461816  0.04735054  1.1559606 ]
[-0.08669887 -0.96032435  0.07046975  1.4631066 ]
[-0.10590536 -1.1562355   0.09973188  1.776945  ]
[-0.12903006 -0.9623683   0.13527077  1.5168618 ]
[-0.14827743 -0.7691172   0.16560802  1.2692816 ]
[-0.16365978 -0.5764511   0.19099365  1.032703  ]
[-0.1751888 -0.7735289  0.2116477  1.3787556]
Episode: 0, total_reward: 14.00
[ 9.0170368e-05  1.6930157e-01  3.1482041e-02 -2.7604392e-01]
[ 0.0034762   0.36396053  0.02596116 -0.5586335 ]
[ 0.01075541  0.16848399  0.01478849 -0.25788572]
[ 0.01412509 -0.02684594  0.00963078  0.03942477]
[ 0.01358817 -0.22210467  0.01041927  0.33513066]
[ 0.00914608 -0.02713254  0.01712188  0.04575157]
[ 0.008603

KeyboardInterrupt: 

: 

In [8]:
agent = DQNAgent(env)
num_episodes = 400

for ep in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train(state, action, next_state, reward, done)
        print(state)
        env.render()
        total_reward += reward
        state = next_state

    print("Episode: {}, total_reward: {:.2f}".format(ep, total_reward))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  if __name__ == "__main__":


[ 0.01057147 -0.04320866 -0.03033668  0.01092238]
[ 0.0097073  -0.2378827  -0.03011823  0.29388148]
[ 0.00494965 -0.4325626  -0.0242406   0.57691544]
[-0.0037016  -0.62733656 -0.01270229  0.8618645 ]
[-0.01624834 -0.82228327  0.00453499  1.1505265 ]
[-0.032694   -1.017464    0.02754553  1.4446281 ]
[-0.05304328 -1.212914    0.05643809  1.745789  ]
[-0.07730156 -1.4086303   0.09135387  2.0554802 ]
[-0.10547417 -1.2145534   0.13246347  1.7923994 ]
[-0.12976523 -1.4108883   0.16831146  2.1231523 ]
Episode: 0, total_reward: 10.00
[-0.03552703 -0.04178202  0.01998061 -0.02786498]
[-0.03636267 -0.23718473  0.01942331  0.27105448]
[-0.04110637 -0.43257838  0.0248444   0.5697998 ]
[-0.04975793 -0.23781352  0.0362404   0.28504613]
[-0.0545142  -0.04322667  0.04194131  0.00400986]
[-0.05537874  0.15126947  0.04202151 -0.27515066]
[-0.05235334  0.34576747  0.0365185  -0.5542894 ]
[-0.045438    0.15015228  0.02543271 -0.2503282 ]
[-0.04243495  0.34490198  0.02042615 -0.5348819 ]
[-0.03553691  0.14

KeyboardInterrupt: 

: 

In [2]:
import sys
print(sys.version)

3.8.9 (default, Apr 13 2022, 08:48:06) 
[Clang 13.1.6 (clang-1316.0.21.2.5)]
